<a href="https://colab.research.google.com/github/traubad/Toxic_Spans_Detection/blob/main/Final_Toxic_Spans_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Toxic Spans Detection </h1>
<h2> Semeval 2020 Task </h2>
<h3> Adam Traub </h3>

In [ ]:
# Imports

# Preprocessing stuff
import re
import spacy
import string
import itertools

import nltk
nltk.download('stopwords')
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from ast import literal_eval

# Structural Stuff
from gensim.models import Word2Vec
import pandas as pd

# ML Stuff
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Misc.
import sys
import numpy as np

#Only Necessary for google colab
from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


<h2> Setting up Libraries post import </h2>

In [ ]:
# Setup Libraries
sp = spacy.load('en_core_web_sm')

# nltk
stopWords = set(stopwords.words('english'))

# colab
drive.mount('/content/drive/')

# Data
data_source = '/content/drive/My Drive/ToxicSpans/tsd_trial.csv'

# General System information
py_version = sys.version.replace('\n',' ')
print("Initial Setup: ")
print(f"\tPython Version: {py_version}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Initial Setup: 
	Python Version: 3.6.9 (default, Oct  8 2020, 12:12:24)  [GCC 8.4.0]


<h2> Utility functions for printing spans, particularly for printing in color</h2>

In [ ]:
# Helper functions mostly for display
class text_color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


def color_string(in_string, color=text_color.RED):
    return f"{color}{in_string}{text_color.END}"


def color_span(span, original_text, color=text_color.RED):
    out_text = original_text
    spans = contiguous_ranges(span)
    bad_words = {out_text[start:end+1] for (start,end) in spans}
    for bw in bad_words:
        out_text = out_text.replace(bw, color_string(bw, color),999)
    return out_text


def contiguous_ranges(span_list):
    """Extracts continguous runs [1, 2, 3, 5, 6, 7] -> [(1,3), (5,7)].
    From: https://github.com/ipavlopoulos/toxic_spans/blob/master/evaluation/fix_spans.py"""
    output = []
    for _, span in itertools.groupby(
        enumerate(span_list), lambda p: p[1] - p[0]):
        span = list(span)
        output.append((span[0][1], span[-1][1]))
    return output

    
def print_row_and_span(i, text, span, color=text_color.RED):
    color_text = color_span(span, text, color).replace('\n','\n    ')
    print(f"{i+1:02}: {contiguous_ranges(span)}\n    {color_text}\n")

<h2> General Options</h2>

In [ ]:
CLASSIFIERS = {
    'random_forest': RandomForestClassifier(n_jobs=2, random_state=0), #Warning: Prohibitively Slow!
    'sgd': SGDClassifier(loss='log'), #simple gradient descent
    'knn': KNeighborsClassifier(n_neighbors=9), #k nearest neighbors - works better than sgd
}

PARSING_OPTIONS = {
    'use_lemmas': True,
    'remove_extra_whitespace': True,
    'remove_punctuation': True,
    'remove_pronouns': True, #Requires use_lemmas
    'remove_stopwords': True, #Requires use_lemmas
    'make_lowercase': True,
    'max_span_length': 60,
    'min_span_length': 3,
    'classifier': CLASSIFIERS['knn'],
}

W2V_PARAMS = {
    'W2V_window': 10,
    'W2V_size': 100, #SDG you can go pretty high with this
    'W2V_min_count': 1, #probably keep this at 1
    'W2V_workers': 4, #probably keep this at 4
}

<h2> Parser </h2>

In [ ]:
def string_to_parsed_list(in_string, options=None):
    if options is None:
        options = PARSING_OPTIONS

    if options['make_lowercase']:
        in_string = in_string.lower()

    if options['remove_extra_whitespace']:
        in_string = " ".join(in_string.split())

    if options.get('remove_punctuation'):
        in_string = re.sub(r'[^\w\s]', '', in_string)

    if options['use_lemmas']:
        out = [word for word in sp(in_string)]

        if options['remove_pronouns']:
            out = filter(lambda word: str(word.lemma_) != "-PRON-", out)

        if options['remove_stopwords']:
            out = filter(lambda word: str(word.lemma_) not in stopWords, out)

        if not options.get('preserve_original', False):
            out = [word.lemma_ for word in out]

    else:
        out = in_string.split(' ')

    if options['min_span_length']:
        out = [word for word in out if len(str(word)) >= PARSING_OPTIONS['min_span_length']]

    return out

<h2> Read the data in and provide some basic information</h2>

In [ ]:
# Read data
data = pd.read_csv(data_source)

# Make spans into list
data["spans"] = data.spans.apply(literal_eval)
original_length = len(data)

# Filter any rows with spans bigger than max_span_length
data = data[data['spans'].map(len) <= PARSING_OPTIONS['max_span_length']]

print(f"Data Loaded, there were originally {original_length} entries")
print(f"After removing all spans with a length greater than {PARSING_OPTIONS['max_span_length']} characters, there are now {len(data)} entries.\n")

for i, row in data.head(5).iterrows():
    print_row_and_span(i, row['text'], row['spans'], text_color.RED)

Data Loaded, there were originally 690 entries
After removing all spans with a length greater than 60 characters, there are now 670 entries.

01: [(15, 19), (27, 31)]
    Because he's a moron and a bigot. It's not any more complicated than that.

02: [(29, 34)]
    How about we stop protecting idiots and let nature add some bleach to the gene pool. We can always submit their names for the Darwin awards.

03: [(166, 171)]
    If people  were  smart, they would  Boycott this  inept  airline,  but   they  are  not  smart,  so   rogue  businesses  like  this  one,   still thrive   taking the idiots  for  a ride...

04: [(87, 92)]
    Trump Claimed that Russia will never invade the Ukraine, when Russia already has - how stupid can people be?

05: []
    As long as your willing to pay a lot more for products you buy, then fine.
    But you better not be going to Costco and Walmart to buy stuff because it's cheaper.
    If so, we get to call you a hypocritical wanker.



<h2>Split the data for testing and training</h2>

In [ ]:
# Train test split (70-30)
train, test = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)

<h2>Functions for getting the toxic and non_toxic words and label them</h2>
<h3>Also where most of the parsing takes place</h2>

In [ ]:
# Creating word train data set
# Getting toxic words
def get_toxic_words(data):
  toxic_string = ''
  comment = data['text']
  prev_idx = -99

  for idx in data['spans']:
    if idx != (prev_idx + 1):
      toxic_string += ' '
      
    toxic_string += comment[idx]
    prev_idx = idx

  return string_to_parsed_list(toxic_string)

# Getting non toxic words
def get_non_toxic_words(data):
  non_toxic_string = ''
  prev_idx = -99

  for idx, com in enumerate(data['text']):
    if idx not in data['spans']:
      if idx == (prev_idx +1):
        non_toxic_string += com
      else:
        non_toxic_string += ' ' + com

      prev_idx = idx

  return string_to_parsed_list(non_toxic_string)

# create the word_data for training and testing
def create_word_data(data):
    toxic_words = train.apply(get_toxic_words, axis=1).values
    toxic_words = list(set([item for sublist in toxic_words for item in sublist])) #Flatten it

    non_toxic_words = train.apply(get_non_toxic_words, axis=1).values
    non_toxic_words = list(set([item for sublist in non_toxic_words for item in sublist]))

    words = non_toxic_words + toxic_words
    labels = np.append(np.zeros(len(non_toxic_words)), np.ones(len(toxic_words)))

    word_data = pd.DataFrame(list(zip(words, labels)), columns=['Words', 'Labels'])

    return word_data

In [ ]:
# create word sets
word_train_data = create_word_data(train)
word_test_data = create_word_data(test)

<h2>W2V embeddings for training data</h2>

In [ ]:
# Put word embedding in word data
def get_word_embedding(word):
  try:
    embedding = model.wv[word]
  except:
    embedding = np.zeros(W2V_PARAMS['W2V_size'])

  return embedding

# Create W2V embeddings
training_comment_words = [string_to_parsed_list(row) for _, row in train['text'].iteritems()]

# Creates numerical representations of text data
model = Word2Vec(sentences=training_comment_words, 
                 size= W2V_PARAMS['W2V_size'], 
                 window= W2V_PARAMS['W2V_window'], 
                 min_count= W2V_PARAMS['W2V_min_count'], 
                 workers= W2V_PARAMS['W2V_workers'])

word_train_data['Vectors'] = word_train_data['Words'].apply(get_word_embedding)
word_test_data['Vectors'] = word_test_data['Words'].apply(get_word_embedding)


In [ ]:
word_test_data.tail(20)

,Words,Labels,Vectors
3166,islamic,1.0,"[-0.0028621643, 0.0043536727, -0.0007164391, 0.0015838945, 0.0047234236, 0.0028461756, -0.002154229, -0.003112106, -0.0019454921, 0.0013121659, 0.003866197, 0.0046383725, -0.00047905263, -0.0034406784, 0.00022200674, 0.0013823613, 0.0006274088, -0.0048103263, -0.0022406883, -9.797256e-05, -0.0023656022, 0.0034154053, 0.0023334194, -0.0007789052, -0.0029026803, -0.0022613995, 0.0025104408, -0.002418102, 0.0017173865, 0.005014782, -0.002665927, 0.00044265378, -0.0005802945, -0.0014338188, 0.0048714103, 0.001570337, -0.0012572918, 0.0034740267, -0.0034886478, -0.0041079717, 0.0010151163, -0.00184437, 0.0021448105, -0.0035009084, 0.0022382652, -0.004266118, -0.0009133386, 0.0020898038, -0.0036836239, -0.0027017111, -0.003571011, -0.0011280326, -0.0039671427, 0.00053747284, 0.004077051, 0.0014892444, 0.0034610506, 0.0013702804, 0.0025870393, -0.003546858, -0.00091649574, 0.003981613, -0.0022639947, 0.0011767818, -0.002057557, -0.00039286818, -0.0045079715, 0.0033587983, -0.003996367, 0.0028331708, 0.0031462598, 0.0032210744, -0.004713818, 0.0002955223, -0.0036015003, -0.0008857186, 0.0012627362, -0.0004130935, -0.0027922562, -0.0032261908, -0.0032241496, 0.0028531633, 0.0046008, -0.0023259197, 0.0058195856, -0.0015388211, -0.00077762356, 0.00014075126, -0.0046495204, -0.002408899, 0.0024491148, 0.00039252313, -0.003522276, -0.0047478913, -0.0006251089, -0.0033334927, -0.0029991653, 0.0048216414, 0.0018350246, -0.0009982756]"
3167,rape,1.0,"[-0.0023681738, 0.004270104, 0.00046689616, -0.0010216752, 0.001001719, -0.0008725737, 0.0046186675, 0.004579107, 0.0010339294, -0.004176913, -0.0043805875, 0.004821485, -0.0043637482, 0.0035740419, 0.0018190277, 0.0012261763, 0.0021079946, 0.0012234831, 0.0008357547, -0.0041445787, 0.003186188, -0.0025788532, 0.0037856766, -0.004658322, -0.0006342563, -0.0025218183, 0.001663197, -0.0019419944, -0.0009568514, 0.003554747, -0.0037047067, -0.0021241934, -0.0043449667, 0.0010266593, -0.004848526, -0.0014186555, 0.0023727168, -0.003387948, -0.0042369864, -0.0036686943, -0.003299002, 0.0019611756, 0.0008307423, 0.0047138915, 0.004338172, -0.0045322357, -0.0028346742, -0.0038743296, 0.0014337479, -0.003279065, 0.003259138, -0.004606722, -0.005026784, 0.0010956711, -0.004744649, -0.001138697, -0.0037304529, -0.00087971095, 0.0036580048, -0.00038486562, -0.003204813, -0.0033170204, -0.0021093646, -0.004105667, 0.0006727908, 0.0035869954, -0.0010005482, -0.003105616, -0.004052837, 0.004370245, -0.004529118, 0.00071398844, -0.0022298752, -0.0044920747, -0.0038450058, -0.003924193, -0.0021931124, 0.0029101684, -0.0023469331, -0.0039206557, -0.0007101661, 0.00043048494, -0.001643447, 0.003765482, -0.002968095, -0.0027054274, -0.001546567, 0.0028314616, 0.0031002872, -0.0013520247, -0.004076317, 0.001185844, 0.00350169, -0.0016041921, 0.0029690466, -0.0038322539, 0.0005121846, 0.0018182007, 0.0004415427, 0.0048511783]"
3168,life,1.0,"[0.0029740266, -0.0050461097, 0.0049759923, -0.0030307763, -0.0010227744, -0.0021292048, -0.0054388735, -0.0043151365, 7.012336e-05, 0.003172196, -0.00051954336, -0.00014361329, -0.0033014917, -0.00072858913, -0.0043873014, -0.003558395, -0.0012333348, 0.0021637848, 0.003155723, 0.00011257008, 0.0022502567, 0.003063488, -0.00327663, -0.00037263634, -0.005237315, 0.0022799163, 0.0051655425, -0.0011916348, -0.0006823736, 0.0035622038, 0.0014717311, -0.0009845953, -0.0023678702, 0.0027083193, -0.003556968, -0.0033825939, -0.0022389835, 2.4573957e-05, -0.0047494993, 0.0046992493, -0.00039438932, 0.0034490358, 0.0013855947, -0.0025891066, -0.00057229935, -0.000894425, -0.0027861148, -0.0038590706, -0.000113365466, -0.0034490847, -0.004171869, 0.0018805754, 5.4559445e-05, 0.0015444162, 0.000903341, -0.0022167033, 0.0034084425, 0.0027352998, 0.00029633468, -0.0011006548, -0.0020788605, -0.003946295, -0.0038432092, -0.00014417284, 0.0019946794, 0.00605191, 0.0044729183, 0.0027464796, 0.003056397, 0.0010854093, 0.0010101815, -0.0034189452, 0.0016933

<h2>Train the model</h2>

In [ ]:
# Train ML model
# Give word embedding and if the word is toxic or not (label)
# If word not in vocabulary, give 0 vector
classifier = PARSING_OPTIONS['classifier']

# word classification - YEAAH
labels = word_train_data['Labels']

# QnD stuff, ignore it. Basically changing data types. Y is effectively is list of list where each inner list is a word embedding 
x = word_train_data['Vectors'].values
y = [list(z) for z in x] 

# use hstack to add additional features

# Do THE THING (training)
classifier.fit(y, labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

<h2>Make Predictions</h2>

In [ ]:
# Predict if word is toxic in sentence
# If toxic, get the indices of the word
# Repeat
result = []
found = 0
not_found = 0

# Each comment
test_comment_word_sets = []
all_bad_words = []

for _, row in test['text'].iteritems():
    test_comment_word_sets.append(string_to_parsed_list(row, 
                                   options={**PARSING_OPTIONS,
                                       **{'preserve_original': PARSING_OPTIONS['use_lemmas']}}))

for index, test_comment_word_set in enumerate(test_comment_word_sets):
  # Each word of comment
  span = []
  bad_word = []

  for word in test_comment_word_set:
      original_word = str(word)
      if PARSING_OPTIONS['use_lemmas']:
          word = str(word.lemma_)
      # Get word embedding 
      try: # if the word exists
        embedding = model.wv[word] # add it's embedding
        found+=1
      except: # if it's a new word
        embedding = np.zeros(W2V_PARAMS['W2V_size']) # it is represented by 0's
        not_found+=1
    
      # Make prediction
      predictor = embedding.reshape(1, -1)
      predictor.setflags(write=True)
      prediction = classifier.predict(predictor)[0]

      # If prediction is toxic, get span, add to span variable
      if prediction == 1: # revisit if the same word is there more than once
        if word in bad_word:
            continue #We've seen this word before, skip it
        bad_word.append(word)
        span_begins = [i for i in range(len(test.iloc[index]['text'])) if test.iloc[index]['text'].lower().startswith(original_word.lower(), i)] 

        for span_begin in span_begins:
            span_end = span_begin + len(original_word)
            for i in range(span_begin, span_end):
                span.append(i)
        
  result.append(span)
  for b in bad_word:
      all_bad_words.append(b.lower())

all_bad_words = set(all_bad_words)

In [ ]:
print(f"Words Found: {found}")
print(f"Words Not Found: {not_found}")
print(f"Bad Words: {all_bad_words}")

Words Found: 2590
Words Not Found: 834
Bad Words: {'airline', 'headline', 'man', 'httpwwwfactcheckorg201601obamasnumbersjanuary2016update', 'monster', 'timely', 'peek', 'dumpy', 'confused', 'heartily', 'rear', 'againanother', 'sentence', 'confuse', 'crp', 'brink', 'venom', 'bloody', 'present', 'nail', 'virtually', 'review', 'monthly', 'idiotic', 'trap', 'favour', 'photo', 'normal', 'mulcair', 'surprisingly', 'ready', 'forth', 'washing', 'wit', 'carlaw', 'suck', 'nkorea', 'dime', 'feature', 'thatya', '1st', 'ethnicity', 'pansy', 'protein', 'commission', 'mayor', 'mileage', 'existential', 'bloodline', 'bye', 'bozell', 'willfully', 'rioter', 'pretend', 'psychotic', 'naturally', 'derange', 'kampf', 'jiberish', 'deficiency', 'cram', 'audience', 'conflate', 'reckless', 'enuf', 'repeatedly', 'eager', '2500', 'schoolwork', 'degraded', 'inauguration', 'speed', 'staff', 'odor', 'opt', 'blow', 'purposely', '147', 'earth', 'surprise', 'bow', 'claw', 'asian', 'sociopath', 'bicyclist', 'socialism', 

<h2>Display raw results</h2>

In [ ]:
pd.set_option('display.max_colwidth', -1)

for i, (row_i, row) in enumerate(test.head(10).iterrows()):
    print_row_and_span(row_i, row['text'], result[i])

374: [(30, 39), (69, 76), (86, 90), (123, 128), (130, 134)]
    She should cut and run. It is ridiculous for her to be carrying that mortgage at this stage - especially when she wants to retire early. She doesn't need the house.

163: [(11, 14)]
    You are an ugly American.

496: [(0, 3), (46, 50)]
    Heck, I remember when you couldn't leave your horse tied up outside without some idiot stealing it.

659: [(8, 17), (20, 24)]
    Are you ILLITERATE, Chuck?

285: [(3, 13), (25, 35), (129, 133)]
    My grandfather must have paraphrased this, he simply taught me, "Never argue with an idiot, you'll never win." Sound words I use often. The main reason I never argue with myself.

375: []
    You gotta hit the Americans where it hurts.

321: [(20, 28), (63, 69), (97, 104), (121, 125), (131, 140), (170, 182), (256, 265), (314, 325), (352, 358), (400, 411), (431, 435), (512, 522)]
    A clueless fool and demagogue like tRump cannot bring it back? Shocker. 
    
    
    Just like his foolish a

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


<h2>Evaluation</h2>

In [ ]:
# f1 = 2*(Recall * Precision) / (Recall + Precision)
def f1(predictions, gold):
   rec = recall(predictions, gold)
   prec = precision(predictions,gold)
   return 0 if (rec + prec == 0) else (2*(rec * prec) / (rec + prec))


def precision(predictions, gold): # TP/TP+FP
    TP = len(set(predictions).intersection(set(gold)))
    FP = len(set(predictions) - set(gold))
    return 0 if (TP+FP==0) else TP / (TP+FP)


def recall(predictions, gold): # TP/TP+FN
    TP = len(set(predictions).intersection(set(gold)))
    FN = len(set(gold) - set(predictions))
    return 0 if (TP+FN==0) else TP / (TP+FN)


def calculate_metric(result, method):
    """
    The above methods will only measure 1 comment,
    This generalized code will run the given method against all comments
    """
    tally = 0.0
    for i, (_, row) in enumerate(test.head(10).iterrows()):
        tally += method(result[i], row['spans'])
    tally /= len(result)
    return tally
    
print(f"f1-score: {calculate_metric(result, f1)}")
print(f"Precision: {calculate_metric(result, precision)}")
print(f"Recall: {calculate_metric(result, recall)}")

f1-score: 0.011794400431858306
Precision: 0.010165935925696219
Recall: 0.01589876703439325
